In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Adrenocortical_Cancer/GSE75415'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene exrpression profiling of childhood adrenocortical tumors"
!Series_summary	"Pediatric adrenocortical tumors (ACT) are rare and often fatal malignancies; little is known regarding their etiology and biology. To provide additional insight into the nature of ACT, we determined the gene expression profiles of 24 pediatric tumors (five adenomas, 18 carcinomas, and one undetermined) and seven normal adrenal glands. Distinct patterns of gene expression, validated by quantitative real-time PCR and Western blot analysis, were identified that distinguish normal adrenal cortex from tumor. Differences in gene expression were also identified between adrenocortical adenomas and carcinomas. In addition, pediatric adrenocortical carcinomas were found to share similar patterns of gene expression when compared with those published for adult ACT. This study represents the first microarray analysis of childhood ACT. Our findings lay the groundwork for establishin

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = None
age_row = None
gender_row = 0  # gender data is available in key 0
convert_trait = None
convert_age = None

# Define conversion functions
def convert_trait(value):
    return None  # No data available for Adrenocortical_Cancer trait

def convert_age(value):
    return None  # No data available for age

def convert_gender(value):
    gender_map = {'female': 0, 'male': 1, 'unknown': None}
    actual_value = value.split(':')[1].strip()
    return gender_map.get(actual_value.lower(), None)

# Save cohort information
save_cohort_info('GSE75415', './preprocessed/Adrenocortical_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction step is skipped because trait_row is None


A new JSON file was created at: ./preprocessed/Adrenocortical_Cancer/cohort_info.json
